# Cleansing customer data


Student: Yue Ling

Year: 2022



# Included Modules and Packages

Code cell containing your includes for modules and packages

In [1]:
import psycopg2
import json
import csv
from datetime import datetime as dt


import math
import numpy as np
import pandas as pd



from IPython.display import display, HTML


from jellyfish import soundex, levenshtein_distance

from fuzzywuzzy import fuzz

from fuzzywuzzy import process as fuzz_process

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  



In [2]:
def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)


cursor = connection.cursor()

def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")
def my_recursive_print_json(j, level = -1):
    "given a json object print it"
    
    level += 1
    
    spaces = "    "
    
    if type(j) is dict:
        dict_2_list = list(j.keys())
        for k in dict_2_list:
            print(spaces * level + k)
            my_recursive_print_json(j[k], level)
            
    elif type(j) is list:
        for (i, l) in enumerate(j):
            print(spaces * level + "[" + str(i) + "]")
            my_recursive_print_json(l, level)
                  
    else:
        print(spaces * level + "value:", str(j))
                  
def my_read_nested_json(file_name):
    "given a file of json, read it and parse it meaningfully"
    
    f = open(file_name, "r")
    
    j = json.load(f)
    
    f.close
    
    my_recursive_print_json(j)
def my_explore_staging_table(table_name):
    "given a table name, explore it"
    
    print("\n---------------------------------------------------")
    print("Exploring Columns for Table:", table_name)
    print("---------------------------------------------------\n")
    
    rollback_before_flag = True
    rollback_after_flag = True
    
    connection.rollback()
    
    query = "select * from " + table_name + " where 0 = 1;"
    
    cursor.execute(query)

    connection.rollback()    

    column_list = [d[0] for d in cursor.description]
    
    for column_name in column_list:
        
        if column_name == "stage_id":
            continue;
        
        print("---------------------------------------------------")
        print("Column:", column_name)
        print("---------------------------------------------------")

        
        query = "select min(length(" + column_name + ")) as min_length, "
        query += " max(length(" + column_name + ")) as max_length, "
        query += " count(*) as total_rows, "
        query += " count(distinct " + column_name + ") as total_distinct_values"
        query += " from " + table_name + ";"
        
        df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
        
        display(HTML(df.to_html()))
        
        query = "select " + column_name + ", count(*) from " + table_name 
        query += " group by " + column_name + " order by 2 desc limit 10;"
        
        df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
        
        display(HTML(df.to_html()))

# 2.5.1 Validate the city, state, and zip for stage_1_peak_customers against the zip_codes table

AGM does not want to give its customer list to 3rd party sales channels, including Peak Delivery.  For that reason, we can expect some variation in customer first and last names, and in the street.  However, the city, state, and zip should be validated by Peak's system, so we do not anticipate any issues.


Sort by stage_id


In [3]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select  *
from stage_1_peak_customers 
where (zip::numeric, city, state) not in (select distinct zip::numeric, 
    city, state from zip_codes)
order by stage_id

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,stage_id,sale_id,customer_id,first_name,last_name,street,city,state,zip


# 2.5.2 Find all customer records in stage_1_peak_customers where any of first_name, last_name, and/or street do not match a customer in the customers table

AGM does not want to give its customer list to 3rd party sales channels, including Peak Delivery.  For that reason, we can expect some variation in customer first and last names, and in the street.  

Sort by stage_id



In [4]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select  *
from stage_1_peak_customers 
where (first_name, last_name, street) not in (select distinct first_name, 
    last_name, street from customers)
order by stage_id

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,stage_id,sale_id,customer_id,first_name,last_name,street,city,state,zip
0,10,5763728768,3729016,Hyrum,Knuckles,86668 Spenser Terrace,Oakland,CA,94618
1,20,5763728877,3728936,Roseann,Coyish,11707 American Ash Ter,Orinda,CA,94563
2,24,5763728428,3729287,Hali,Ducker,8 Orion Pass,El Cerrito,CA,94530
3,26,5763728393,3728674,Melantha,Golborn,6140 North Field Alley,Orinda,CA,94563
4,36,5763729212,3729191,Eleni,Jansen,66 Bartelt Hill,Oakland,CA,94607
5,40,5763729129,3728856,Clyve,Humonds,22 Brent Wood Hill,Berkeley,CA,94709
6,51,5763728864,3729178,Rutledge,Hellwing,606 Gulf Plz,El Cerrito,CA,94530
7,60,5763729313,3728402,Kalli,Kemel,18373 Golf View Pass,Berkeley,CA,94702
8,72,5763728980,3729213,Honina,Philson,28 Clarendon Plaza,Berkeley,CA,94702
9,73,5763728921,3729194,Nicky,Haley,88424 Warrior Lane,Oakland,CA,94602


# 2.5.3 Find the percentage of Peak's customer records that do not match to AGM's customers table

Write a query to find the percentage of Peak's customer records that do not match AGM's.  The percentage can be found by taking the number of customer records in stage_1_peak_customers that do not match and dividing by the number of customers records in stage_1_peak_customers and multiplying by 100.


## columns not matched in agm's table:

* columns that don't make sense, only 1 of two tables have: stage_id, closest_store_id, distance
* columns that were queries before that matched: zip, city, state

In [5]:
rollback_before_flag = True
rollback_after_flag = True

query = """
with a as(
    select  count(*) as count_mismatched
    from stage_1_peak_customers 
    where (first_name, last_name, street) not in (select distinct first_name, 
        last_name, street from customers)
    )

select CAST((CAST((select count_mismatched from a) AS FLOAT) / CAST( count(*) AS FLOAT) ) * 100 as numeric(6,2)) as perc_mismatched, 
    count(*) as total_customers_record, (select count_mismatched from a) ,
    (CAST( count(*) AS FLOAT) -  CAST((select count_mismatched from a) AS FLOAT)) as count_matched
from stage_1_peak_customers

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,perc_mismatched,total_customers_record,count_mismatched,count_matched
0,13.4,97,13,84


# 2.5.4 Executive summary on customer data

The executives are looking for recommendation of one of the following:
* Continue to withhold the customer data from 3rd party sales channels
* Give customer data to 3rd party sales channels

Recommend exactly one of these.

Support you recommendation with an explanation based on what you have seen from this preliminary data load.


## Recommendation
I recommend that we continue to hold our customer data to ourselves to keep our trade secret advantage. In the case when Peak one day decides to go into the meal-kit service, we would not give them a list of potential customers – in direct competion with AGM. As the current preliminary data shows, and if this trend continues, we only have 13% of mismatched customers' first name, last name and/or street records between Peak's and our own customer records. We can still gather insights about the sales on the remaining 84% of customer sales.  

In [6]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select  *
from stage_1_peak_customers 
where (first_name, last_name, street) not in (select distinct first_name, 
    last_name, street from customers)
order by stage_id

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,stage_id,sale_id,customer_id,first_name,last_name,street,city,state,zip
0,10,5763728768,3729016,Hyrum,Knuckles,86668 Spenser Terrace,Oakland,CA,94618
1,20,5763728877,3728936,Roseann,Coyish,11707 American Ash Ter,Orinda,CA,94563
2,24,5763728428,3729287,Hali,Ducker,8 Orion Pass,El Cerrito,CA,94530
3,26,5763728393,3728674,Melantha,Golborn,6140 North Field Alley,Orinda,CA,94563
4,36,5763729212,3729191,Eleni,Jansen,66 Bartelt Hill,Oakland,CA,94607
5,40,5763729129,3728856,Clyve,Humonds,22 Brent Wood Hill,Berkeley,CA,94709
6,51,5763728864,3729178,Rutledge,Hellwing,606 Gulf Plz,El Cerrito,CA,94530
7,60,5763729313,3728402,Kalli,Kemel,18373 Golf View Pass,Berkeley,CA,94702
8,72,5763728980,3729213,Honina,Philson,28 Clarendon Plaza,Berkeley,CA,94702
9,73,5763728921,3729194,Nicky,Haley,88424 Warrior Lane,Oakland,CA,94602


# Soundex to compare the similar sounding first and last names (just small misspelling)

Matching the 2 tables' customer information based on zip, city, and state, 6 of the 13 mismatched records are closesly aligning with AGM's records when using a soundex function on the mismatched names. As shown in the follwing query, the mismatches are mostly small misspelling errors, but same sounding first and last name. 

In [7]:
# soundex comparison on all first and last name from agm's customers and compare 
# with peak's customer records

rollback_before_flag = True
rollback_after_flag = True
query = """

select p.stage_id as peak_stage_id, c.first_name as agm_first_name, c.last_name as agm_last_name,
    p.first_name as peak_first_name, p.last_name as peak_last_name
from customers as c
     join stage_1_peak_customers as p
         on p.zip = c.zip and c.city = p.city and c.state = p.state
where (p.first_name, p.last_name) not in (select distinct first_name, 
    last_name from customers) and (soundex(p.first_name) = soundex(c.first_name)
    and soundex(p.last_name) = soundex(c.last_name))
order by stage_id
;

"""
my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)


,peak_stage_id,agm_first_name,agm_last_name,peak_first_name,peak_last_name
0,10,Hiram,Knuckles,Hyrum,Knuckles
1,24,Halli,Ducker,Hali,Ducker
2,40,Clyve,Hammonds,Clyve,Humonds
3,60,Kalli,Kemell,Kalli,Kemel
4,73,Nickey,Haley,Nicky,Haley
5,97,Eula,Deware,Eula,Dewair


# Soundex misclassified 

As we can see in the following query, peak_stage_id 60 and 72 showed up twice, this is a suffer from Soundex misclassification where same sounding first names or last names of different people may be wrongly classified as potential matches. However, this misclassification is rare and fairly easy to spot out. 

In [8]:
# soundex comparison on all first and last name from agm's customers and compare 
# with peak's customer records

rollback_before_flag = True
rollback_after_flag = True
query = """

select p.stage_id as peak_stage_id, c.first_name as agm_first_name, c.last_name as agm_last_name,
    p.first_name as peak_first_name, p.last_name as peak_last_name
from customers as c
     join stage_1_peak_customers as p
         on p.zip = c.zip and c.city = p.city and c.state = p.state
where (p.first_name, p.last_name) not in (select distinct first_name, 
    last_name from customers) and (soundex(p.first_name) = soundex(c.first_name)
    or soundex(p.last_name) = soundex(c.last_name))
order by stage_id
;

"""
my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)


,peak_stage_id,agm_first_name,agm_last_name,peak_first_name,peak_last_name
0,10,Hiram,Knuckles,Hyrum,Knuckles
1,24,Halli,Ducker,Hali,Ducker
2,40,Clyve,Hammonds,Clyve,Humonds
3,60,Kalli,Kemell,Kalli,Kemel
4,60,Kylie,Saxelby,Kalli,Kemel
5,72,Junina,Philson,Honina,Philson
6,72,Ingar,Pilkington,Honina,Philson
7,73,Nickey,Haley,Nicky,Haley
8,75,Israel,O'Dunniom,I,Odunniom
9,78,Klemens,MacAloren,Clemens,McAloren


# Using Fuzzy Logic to find misspelled addresses

We are able to visually inspect the mismatched addresses and see the differences  may just be attributed to typos or shorthands (plz for plaza). To further back up this hypothesis, I used Fuzzy Logic and observe that 4 of the 7 mismatched addresses seem like just typos. The fuzzy scores of 85, 97, 93, 92 are all fairly close to ideal score of 100 for perfect match. This means that the users probably added a space, forgot a character or used an abbreviation. 

In [9]:
# fuzzlogic comparison of the mismatched street between peak and agm's
# customers' street records


rollback_before_flag = True
rollback_after_flag = True
query = """

with mismatched as (select  *
from stage_1_peak_customers 
where (first_name, last_name, street) not in (select distinct first_name, 
    last_name, street from customers)
order by stage_id)

select p.stage_id as peak_stage_id, c.first_name as agm_first_name,
    c.last_name as agm_last_name, c.street as agm_street,
    p.first_name as peak_first_name, p.last_name as peak_last_name, p.street as peak_street
from customers as c
     join stage_1_peak_customers as p
         on p.zip = c.zip and c.city = p.city and c.state = p.state
where (p.street not in (select distinct street from customers)) and 
    c.first_name in (select first_name from mismatched) and
    c.last_name in (select last_name from mismatched)
order by stage_id
;

"""
df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
for i in range(df.shape[0]):
    print("Fuzzy: ratio:", fuzz.ratio(df.agm_street[i], df.peak_street[i]))
    print('agm street: ', df.agm_street[i])
    print('peak street: ', df.peak_street[i], '\n')
    
df

Fuzzy: ratio: 37
agm street:  6140 Northfield Alley
peak street:  11707 American Ash Ter 

Fuzzy: ratio: 92
agm street:  11707 American Ash Terrace
peak street:  11707 American Ash Ter 

Fuzzy: ratio: 38
agm street:  606 Golf Plaza
peak street:  8 Orion Pass 

Fuzzy: ratio: 33
agm street:  11707 American Ash Terrace
peak street:  6140 North Field Alley 

Fuzzy: ratio: 93
agm street:  6140 Northfield Alley
peak street:  6140 North Field Alley 

Fuzzy: ratio: 97
agm street:  6 Bartelt Hill
peak street:  66 Bartelt Hill 

Fuzzy: ratio: 85
agm street:  606 Golf Plaza
peak street:  606 Gulf Plz 



,peak_stage_id,agm_first_name,agm_last_name,agm_street,peak_first_name,peak_last_name,peak_street
0,20,Melantha,Golborn,6140 Northfield Alley,Roseann,Coyish,11707 American Ash Ter
1,20,Roseann,Coyish,11707 American Ash Terrace,Roseann,Coyish,11707 American Ash Ter
2,24,Rutledge,Hellwing,606 Golf Plaza,Hali,Ducker,8 Orion Pass
3,26,Roseann,Coyish,11707 American Ash Terrace,Melantha,Golborn,6140 North Field Alley
4,26,Melantha,Golborn,6140 Northfield Alley,Melantha,Golborn,6140 North Field Alley
5,36,Eleni,Jansen,6 Bartelt Hill,Eleni,Jansen,66 Bartelt Hill
6,51,Rutledge,Hellwing,606 Golf Plaza,Rutledge,Hellwing,606 Gulf Plz


# Executive Summary 
The relatively small 13.4% of mismatches between Peak's customers records compared to AGM's records are primarily mispelling of names, using abbreivations or accidental inserting or deleting a character. With eyeballing and a little data cleaning and mismatch probing with Fuzzy Logic and Soundex, these mismatches can be reduced to 3%. 

This 13.4% of bad data's significance further decreases as we collect more and more data from the trial. This preliminary analysis is done only on 97 records collected in 1 day. If we run the trial for 3 months, we would have about 9000 records – assuming the same amount of Peak's food ordered per day for the rest of 3 months. 

## Bottomline
Ultimately, even if nothing is changed on Peak's data collection, AGM top quality data scientists don't do any data cleaning, and the trend of mismatched records remain the same of 13.4%. This is not a huge issue as AGM is still able to garner insight from the remaining 84%.  Therefore, we should continue to withhold our customer data as trade secret and prevent potential competitors from getting our customers information. 